In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Workspaces/gpt_lite')

In [3]:
import torch
import torch.optim as optim
from utils import *

# <b> GPT-Lite

In [4]:
# Set seed for reproducibility.
seed = 1
set_seed(seed)

# Load GPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Loaded Device: ', device)

Loaded Device:  cuda:0


### <b> Explore Input

In [5]:
# Explore input
with open('input_seuss.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print(f'Total number of tokens: {len(text)}')

Total number of tokens: 30504


In [7]:
print(text[:500])

The Cat in the Hat

By Dr. Seuss

The sun did not shine.
It was too wet to play.
So we sat in the house
All that cold, cold, wet day.

I sat there with Sally.
We sat there, we two.
And I said, "How I wish
We had something to do!"

Too wet to go out
And too cold to play ball.
So we sat in the house.
We did nothing at all.

So all we could do was to

Sit!
Sit!
Sit!
Sit!

And we did not like it.
Not one little bit.

BUMP!

And then
something went BUMP!
How that bump made us jump!

We looked!
Then w


### <b> Hyperparameters

In [8]:
# Define data loading parameters.
batch_size = 64 # 64 # 16
context_size = 256 # 256 # 32
max_iterations = 5000
eval_interval = 500 # 500 # 100
learning_rate = 3e-4 # 3e-4 # 1e-3
eval_iters = 200
n_embeddings = 384 # 384 # 64
n_heads = 6 # 6 # 4
n_layers = 6 # 6 # 4
dropout = 0.2 # 0.2 # 0.0
train_test_split = 0.9

### <b> Load Data

In [9]:
vocab_size, encode, decode, train_data, val_data = data(text, train_test_split)

### <b> Initiate GPT

In [10]:
# Initiate GPT.
model = BigramModel(vocab_size, n_embeddings, n_heads, n_layers, context_size, dropout, device)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

10.788929 M parameters


### <b> Model Training

In [11]:
# Define loss function and optimizer.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Train model and return results.
# Note: If early stopping is False then patience is not used.
model = train_model(
                        train_data, val_data,
                        batch_size = batch_size,
                        context_size = context_size,
                        max_iterations = max_iterations,
                        eval_interval = eval_interval,
                        learning_rate = learning_rate,
                        eval_iters = eval_iters,
                        n_embeddings = n_embeddings,
                        n_heads = n_heads,
                        n_layers = n_layers,
                        dropout = dropout,
                        train_test_split = train_test_split,
                        model = model,
                        optimizer = optimizer,
                        device = device
                        )


step 0: train loss 4.3583, val loss 4.3480
step 500: train loss 1.2170, val loss 2.2679
step 1000: train loss 0.1854, val loss 3.1624
step 1500: train loss 0.0743, val loss 3.8812
step 2000: train loss 0.0567, val loss 4.2883
step 2500: train loss 0.0497, val loss 4.5324
step 3000: train loss 0.0446, val loss 4.6826
step 3500: train loss 0.0424, val loss 4.7823
step 4000: train loss 0.0399, val loss 4.9905
step 4500: train loss 0.0386, val loss 5.0220
step 4999: train loss 0.0378, val loss 5.1313


### <b> Generate Text

In [17]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
output = decode(model.generate(context, max_new_tokens=2000, context_size=context_size)[0].tolist())
print(output)

with open('output_seuss.txt', 'w') as f:
    f.write(output)


I can't say.

Luke Luck likes duck likes lakes.
Luke Luck licks lakes.
Luck's duck licks lakes.

Duck takes licks in lakes Luke Luke Luck likes.
Luke Luck takes licks in lakes duck likes.

I can't blab such blibber blubber!
My tongue isn't make of rubber.

Mr. Knox.  Now come now.  Come now.
You don't have to be so dumb now....

Try to say this, Mr. Knox, please.....

Through three cheese trees three free fleas flew.
While these fleas flew, freezy breeze blew.
Freezy brothereeze made these three trees' cheese freeze.
That's what made these three free fleas sneeze.

Stop it!  Stop it!
That's enough, sir.
I say can't such sill stuff, sir.

Very well, then, Mr. Knox, sir.

Let's have a little talk about tweetle th beetles....

What do you know about tweetle beetles?  Well....

When tweetle beetles fight, 
it's called a tweetle beetle battle.

And they battle in a puddle, 
it's a tweetle beetle puddle battle.

AND So whe saw twee them cat in the him out,
Sall the say!"

"The we the cat wi